change views and favorers to per unit time values

In [ ]:
from datetime import datetime
start = listings['original_creation_tsz'].tolist()
end = listings['ending_tsz'].tolist()
d_start = [datetime.strptime(datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'), "%Y-%m-%d %H:%M:%S") for i, x in enumerate(start)]
d_end = [datetime.strptime(datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'), "%Y-%m-%d %H:%M:%S") for i, x in enumerate(end)]
difference = [(d_end[i] - d_start[i]).days for i in range(len(d_start))]
listings = listings.assign(days=pd.Series(difference))
views = listings['views'].tolist()
num_favorers = listings['num_favorers'].tolist()
days = listings['days'].tolist()
views_days = [views[i] / days[i] for i in range(len(views))]
num_favorers_days = [num_favorers[i] / days[i] for i in range(len(num_favorers))]
listings = listings.assign(views=pd.Series(views_days))
listings = listings.assign(num_favorers=pd.Series(num_favorers_days))
listings.to_pickle("./listings.pkl")

start here

In [1]:
import sys
from tqdm import tqdm
from time import time
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import nltk
from nltk import word_tokenize  
from nltk.stem import WordNetLemmatizer 
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.externals import joblib
nltk.download('punkt')
nltk.download('wordnet')

%matplotlib inline
pd.set_option('display.max_columns', 50)

[nltk_data] Downloading package punkt to /home/mc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/mc/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Open listings and all_features pickle structures

In [2]:
with open('listings.pkl', 'rb') as handle:
    listings = pickle.load(handle)

with open('all_features.pickle', 'rb') as handle:
    all_features = pickle.load(handle)

Vectorizer features

In [ ]:
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

vectorizer = CountVectorizer(min_df=0.001, analyzer='word', tokenizer=LemmaTokenizer(), binary=True)
X = vectorizer.fit_transform(all_features)
X_names = vectorizer.get_feature_names()
print(len(X_names))

In [ ]:
vectorizer = CountVectorizer(analyzer='word', min_df=0.001, tokenizer=nltk.word_tokenize, binary=True)
X = vectorizer.fit_transform(all_features)
X_names = vectorizer.get_feature_names()
print(len(X_names))

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(all_features)
X_names = vectorizer.get_feature_names()
print(len(X_names))

In [ ]:
vectorizer = TfidfVectorizer(tokenizer=nltk.word_tokenize)
X = vectorizer.fit_transform(all_features)
X_names = vectorizer.get_feature_names()
print(len(X_names))

In [ ]:
X_names

Split data into train and test sets

In [ ]:
prices = np.nan_to_num(listings['price'].tolist())
views = np.nan_to_num(listings['views'].tolist())
favorers = np.nan_to_num(listings['num_favorers'].tolist())
test_size = 0.1

X_price_train, X_price_test, y_price_train, y_price_test = train_test_split(X, prices, test_size=test_size)
X_views_train, X_views_test, y_views_train, y_views_test = train_test_split(X, views, test_size=test_size)
X_favorers_train, X_favorers_test, y_favorers_train, y_favorers_test = train_test_split(X, favorers, test_size=test_size)

Make some linear regression models

In [ ]:
clf_price_lasso = linear_model.Lasso(alpha=0.1)
y_price_pred = clf_price_lasso.fit(X_price_train, y_price_train).predict(X_price_test)
print("\nprice prediction")
print("explained variance score:",explained_variance_score(y_price_test, y_price_pred))
print("mean absolute error:",mean_absolute_error(y_price_test, y_price_pred))
print("r2 score:",r2_score(y_price_test, y_price_pred))

In [ ]:
clf_views_lasso = linear_model.Lasso(alpha=0.1)
y_views_pred = clf_views_lasso.fit(X_views_train, y_views_train).predict(X_views_test)
print("\nviews prediction")
print("explained variance score:",explained_variance_score(y_views_test, y_views_pred))
print("mean absolute error:",mean_absolute_error(y_views_test, y_views_pred))
print("r2 score:",r2_score(y_views_test, y_views_pred))

In [ ]:
clf_favorers_lasso = linear_model.Lasso(alpha=0.1)
y_favorers_pred = clf_favorers_lasso.fit(X_favorers_train, y_favorers_train).predict(X_favorers_test)
print("\nfavorers prediction")
print("explained variance score:",explained_variance_score(y_favorers_test, y_favorers_pred))
print("mean absolute error:",mean_absolute_error(y_favorers_test, y_favorers_pred))
print("r2 score:",r2_score(y_favorers_test, y_favorers_pred))

Do clustering

In [4]:
n_features = 1000 # Number of features
n_samples = 36570 # Number of samples
n_components = 100 # Number of components
n_top_words = 10  # Number of top words

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [5]:
# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(all_features)
print("done in %0.3fs." % (time() - t0))

Extracting tf-idf features for NMF...
done in 0.655s.


In [6]:
# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')
t0 = time()
tf = tf_vectorizer.fit_transform(all_features)
print("done in %0.3fs." % (time() - t0))

Extracting tf features for LDA...
done in 0.595s.


In [ ]:
joblib.dump(tfidf, 'tfidf_features.pkl')
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')
joblib.dump(tf, 'tf_features.pkl')
joblib.dump(tf_vectorizer, 'tf_vectorizer.pkl')

In [ ]:
# Fit the NMF model
print("Fitting the NMF model (Frobenius norm) with tf-idf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          alpha=.1, l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

In [ ]:
joblib.dump(nmf, 'tfidf_nmf.pkl')

In [ ]:
print("\nTopics in NMF model (Frobenius norm):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

In [ ]:
print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

In [ ]:
joblib.dump(lda, 'tf_lda.pkl')

In [ ]:
print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

try out gradient boosting regression

In [43]:
from sklearn.metrics import explained_variance_score

In [7]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

prices = np.nan_to_num(listings['price'].tolist())
views = np.nan_to_num(listings['views'].tolist())
favorers = np.nan_to_num(listings['num_favorers'].tolist())
test_size = 0.1

tfidf_train, tfidf_test, y_train, y_test = train_test_split(
    tfidf, prices, test_size = test_size)

In [ ]:
# Fit regression model
params = {'n_estimators': 1000, 'max_depth': 8, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
clf = GradientBoostingRegressor(**params)

clf.fit(tfidf_train, y_train)
mae = mean_absolute_error(y_test, clf.predict(tfidf_test))
print("MAE: %.4f" % mae)

In [42]:
explained_variance_score(y_test, )

0.6114017011620319

In [ ]:
# Fit regression model another version
params = {'n_estimators': 2000, 'max_depth': 10, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
clf_price = GradientBoostingRegressor(**params)

clf_price.fit(tfidf_train, y_train)
mae = mean_absolute_error(y_test, clf_price.predict(tfidf_test))
print("MAE: %.4f" % mae)

In [ ]:
r2 = r2_score(y_test, clf_price.predict(tfidf_test))
print("R2: %.4f" % r2)

Save price regressor model to pickle

In [ ]:

joblib.dump(clf_price, 'tfidf_price_GradientBoostingRegressor.pkl')

In [ ]:
test_clf = joblib.load('tfidf_price_GradientBoostingRegressor.pkl') 

In [ ]:
mae = mean_absolute_error(y_test, test_clf.predict(tfidf_test))
print("MAE: %.4f" % mae)

Make views regressor model

In [8]:
tfidf_train, tfidf_test, y_train, y_test = train_test_split(
    tfidf, views, test_size = test_size)

# Fit regression model
params = {'n_estimators': 1000, 'max_depth': 10, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
clf_views = GradientBoostingRegressor(**params)

clf_views.fit(tfidf_train, y_train)
mae = mean_absolute_error(y_test, clf_views.predict(tfidf_test))
print("MAE: %.4f" % mae)

MAE: 0.6590


In [9]:
joblib.dump(clf_views, 'tfidf_views_GradientBoostingRegressor.pkl')

['tfidf_views_GradientBoostingRegressor.pkl']

In [10]:
test_clf = joblib.load('tfidf_views_GradientBoostingRegressor.pkl') 

In [11]:
mae = mean_absolute_error(y_test, test_clf.predict(tfidf_test))
print("MAE: %.4f" % mae)

MAE: 0.6590


Make num favorers regressor model

In [13]:
tfidf_train, tfidf_test, y_train, y_test = train_test_split(
    tfidf, favorers, test_size = test_size)

# Fit regression model
params = {'n_estimators': 1000, 'max_depth': 10, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
clf_num_favorers = GradientBoostingRegressor(**params)

clf_num_favorers.fit(tfidf_train, y_train)
mae = mean_absolute_error(y_test, clf_num_favorers.predict(tfidf_test))
print("MAE: %.4f" % mae)

MAE: 0.0776


In [14]:
joblib.dump(clf_num_favorers, 'tfidf_num_favorers_GradientBoostingRegressor.pkl')

['tfidf_num_favorers_GradientBoostingRegressor.pkl']

In [15]:
test_clf = joblib.load('tfidf_num_favorers_GradientBoostingRegressor.pkl') 

In [16]:
mae = mean_absolute_error(y_test, test_clf.predict(tfidf_test))
print("MAE: %.4f" % mae)

MAE: 0.0776


In [ ]:
cluster = lda.fit_transform(tf)

In [ ]:
predict = lda.transform(tf[0])

In [ ]:
print(all_features[0])
print(tf[0])
predict = lda.transform(tf[0])
print(predict)

In [ ]:
topic_list = lda.transform(tf_vectorizer.transform([all_features[0]]))[0]

In [ ]:
topic_list.argmax()

In [ ]:
lda.components_[24]

In [ ]:
def print_top_words(model, feature_names, n_top_words, topic_group):
    for topic_idx, topic in enumerate(model.components_):
        if (topic_idx == topic_group):
            message = "Topic #%d: " % topic_idx
            #message += " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
            result = " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
            message += result
            print(message)
    print()
    return(result)

In [ ]:
test_top_words = print_top_words(lda, tf_feature_names, 50, 24)
print(test_top_words.split())

In [ ]:
print(all_features[0].split())

In [ ]:
list(set(all_features[0].split()).intersection(set(test_top_words.split())))

In [ ]:
def top_ten_features(model_features, test_features):
    new_features = 0
    index = 0
    new_features_list = []
    while new_features < 10:
        if model_features[index] != test_features[index]:
            new_features += 1
            new_features_list.append(model_features)

In [ ]:
print_top_words(lda, tf_feature_names, 10)